In [ ]:
#!/usr/bin/env python

"""
DESCRIPTION

    Script for scraping process parameters and defect parameters for PCA
    Outfitted for use with 'process_parameters.csv'
"""

from __future__ import division
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys, os, glob
import numpy as np
import json

def scrape_data(plate_num=['2'], feat_list=None):

    # Retrieve all build features
    build_data = np.genfromtxt('process_parameters.csv', delimiter=',',
                                    dtype=None)
    
    # Obtain build features of interest
    if feat_list==None:
        print build_data[0,:]
        feats = raw_input('Please enter build features as a list of strings: ')
    else:
        pass 
    
    # Iterate through .json files for defect parameters
    processed_parts = []
    response = []
    for f in glob.glob('*.json'):
        data = json.loads(open(f, 'rb').read())
        ifile = os.path.basename(f)
        processed_parts.append(os.path.splitext(ifile)[0][:3]) 
        
        # Obtain user specified defect parameter for regression
        print data.keys()
        defect = raw_input('Please enter (void specific) response variable as a string: ')
        stat = raw_input('Please enter ')
        if defect=='nearest neighbor distance':
            response.append(data[defect]['values'])
        elif defect==''
